In [ ]:
from collections import Counter

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# avoid burning my eyes @ night
plt.style.use("dark_background")

In [ ]:
FILE = "data/survey_results_public.csv"
so_df = pd.read_csv(FILE)

print(so_df.keys())
so_df.describe()

# check for people who aren't paying attention
count_not_apple =  (so_df["Check"] != "Apples").sum()
print(count_not_apple)
print(so_df.shape)
assert(count_not_apple == 0)
# print(so_df[:3])


In [ ]:
# draw count plot of developers based on age

def visualize_devs(df, lang, key="Age",):
    plt.figure()
    plt.xticks(rotation=45)
    # from:
    # print(df[key].unique())
    order =  ['Under 18 years old', '18-24 years old',  \
              '25-34 years old','35-44 years old',\
              '45-54 years old', '55-64 years old',  \
              '65 years or older', 'Prefer not to say']
    sb.countplot(x=key, data=df, order=order)
    title="Ages of %s Programmers" % lang
    plt.title(title)
    filename= "images/%s-of-%s-programmers.png" % (key, lang)
    plt.savefig(filename, bbox_inches="tight")

def get_lang_devs(df, lang):
    col = "LanguageHaveWorkedWith"
    # will not work for single character languages (C, R)
    # will mangle Java and JavaScript, Python and MicroPython
    return df[ df[col].str.contains(lang, na=False) ] 

def get_c_devs(df, lang="C"):
    key = "LanguageHaveWorkedWith"
    cdevs = []
    for index, dev in df.iterrows():
        try:
            # split string into list
            langs_used = dev[key].split(';')
            if lang in langs_used:
                cdevs.append(dev)
        except AttributeError:
#            print(dev[key])
            pass
    return pd.DataFrame(cdevs)

In [ ]:

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import StratifiedShuffleSplit
import traceback
import numpy as np

# still haven't come up with a name
class Foo:
    def __init__(self, dataset, language, jobs=None, n_rich_outliers=0, n_poor_outliers=0, country="United States of America"):
        self.devs   = None
        self.canvas = None
        self.language = language
        self.country = country
        # focus on people who have given ...
        key  = "ConvertedCompYearly"
        key2 = "YearsCodePro"
        df   = dataset.dropna(subset=[key, key2])
        self.key  = key
        self.key2 = key2

        criteria = {"MainBranch":"I am a developer by profession"}

        #print(df["Country"].unique)
        if country:
            criteria["Country"] = country
        for k in criteria:
            df = df[df[k] == criteria[k] ] 

        # chatgpt tells me about filtering with multiple strings
        if jobs:
            df = df[df.isin(jobs).any(axis=1)]

        devs = None
        if len(language) > 1:
            devs = get_lang_devs(df, language)
        else:
            devs = get_c_devs(df, lang=language)
        replacement_dict = {
            'Less than 1 year': '0.5',
            'More than 50 years': '51',
        }

        # https://stackoverflow.com/questions/47443134/update-column-in-pandas-dataframe-without-warning
        pd.options.mode.chained_assignment = None  # default='warn'
        new_column = devs[key2].replace(replacement_dict)
        devs[key2] = pd.to_numeric(new_column, errors='coerce')
        pd.options.mode.chained_assignment = 'warn'  # default='warn'
        # print( devs[key2].unique() )
        
        indices  = devs[key].nlargest(n_rich_outliers).index
        devs = devs.drop(indices)
        indices  = devs[key].nsmallest(n_poor_outliers).index
        self.devs = devs.drop(indices)
        del devs, new_column, criteria
    
    def visualize(self, n_lowest=0, hue="Country"):    
        self.canvas = plt.figure()
        key  = self.key
        key2 = self.key2

        if n_lowest > 0:
            # chatgpt draws my line
            # Calculate the lowest nth point (for example, the 5th lowest value)
            # iloc[-1] gets the last element from the n smallest
            lowest_nth = self.devs[key].nsmallest(n_lowest).iloc[-1]  
            # Draw a horizontal line at the lowest nth point
            # label=f'Lowest {n_poorest}th Point: {lowest_nth_value:.2f}'
            plt.axhline(y=lowest_nth, color='purple', linestyle='--', label="y=%0.2f" % lowest_nth )

        sb.scatterplot(data=self.devs, x=key2, y=key, hue=hue)
        plt.legend(loc='lower center', bbox_to_anchor=(1.5,0)) 
        title = "Annual Salary of %s Developers Over Years of Experience" % self.language\
                + "\nsample size=%i" %  len (self.devs)\
                + "\ncountry=%s" % self.country
        plt.title(title)

    def run_regression(self, split=train_test_split, 
                       x_transform=None, change_base=None, x_shift=0,
                       line_color='red'):
        df = self.devs # .sort_values(by = self.key2)
#        df['binned'] = pd.qcut(df[self.key], q=4, labels=False)
        X = df[self.key2].to_frame() + x_shift
        if x_transform is not None and change_base is not None:
            X = x_transform (X, a=change_base ) 
        elif x_transform is not None:
            X = x_transform (X)   

        y = df[self.key].to_frame()
#        y = df['binned']
    
        X_train, X_test, y_train, y_test = split(X, y, test_size=0.2, random_state=999)

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        print("+----------------------+")
        print("coefficient =", model.coef_)
        print('intercept=', model.intercept_)
        rmse = root_mean_squared_error(y_test, y_pred)
        print("rmse = ", rmse)
        print("sample predictions:")
        print(y_pred[3:6])
        print("+----------------------+")
    
        plt.figure(self.canvas)
        plt.xlim(left=0, right=40)  # Adjust these values as needed
        plt.plot(X_test, y_pred, color=line_color, label='Regression Line')
        plt.legend(loc='lower center', bbox_to_anchor=(1.5,0)) 
        del y_pred, model


    def export_image(self, filename = "images/programmers-%s-%s.png"):
        plt.figure(self.canvas)
        plt.savefig(filename % (self.language, self.country), bbox_inches='tight')

# the higher a is, the steeper the line gets
def log_base_a(x, a=1.07):
    return np.log10(x)/np.log(a)

In [ ]:


# expected python jobs
pyjobs = ["Data scientist or machine learning specialist",
          "Data or business analyst",
          "Data engineer",
#        "DevOps specialist",
#        "Developer, QA or test"
]

python = Foo(so_df, "Python", jobs=pyjobs, n_rich_outliers=9, n_poor_outliers=2)
python.visualize(hue="DevType")
# earnings vary widely after the first year
python.run_regression( x_transform=log_base_a, x_shift=1)
python.run_regression( x_transform=log_base_a, change_base=1.2, x_shift=1, line_color='magenta')
python.run_regression( x_transform=log_base_a, change_base=1.12, x_shift=1, line_color='lightgreen')
python.export_image()

In [ ]:
# expected C jobs
cjobs = ["Developer, embedded applications or devices", 
         "Developer, game or graphics",
         "Hardware Engineer" ,
 #        "Project manager", 
 #        "Product manager"
]
c = Foo(so_df, "C", jobs=cjobs, n_rich_outliers=11)
c.visualize(n_lowest=3, hue="DevType")
c.run_regression(x_transform=log_base_a, change_base=1.25)
c.export_image()

In [ ]:

jsjobs = ["Developer, full-stack",
          "Developer, front-end",
          "Developer, mobile"
]

js = Foo(so_df, "JavaScript", jobs=jsjobs, n_rich_outliers=6, country="Ukraine")
js.visualize(hue="DevType")
js.export_image()

In [ ]:
visualize_devs( get_c_devs(so_df) , "C")

for lang in ["Cobol", "Prolog", "Ada", "Python"]:
    foo = get_lang_devs(so_df, lang)
    visualize_devs(foo, lang)

In [ ]:
# get popularity of different programming languages

#keys re: languages are:
#LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,LanguageDesired

# draw as strip chart
# https://seaborn.pydata.org/generated/seaborn.stripplot.html#seaborn.stripplot

def get_langs(dataset, key="LanguageHaveWorkedWith"):
    lang_count = Counter()
    assert(key in dataset.keys())
    for response in dataset[key]:
        if type(response) == str:
            lang_count.update(response.split(';'))
    langs_by_popularity = dict(
        sorted(lang_count.items(), key=lambda item: item[1], reverse=True)
    )
    return langs_by_popularity

def visualize_langs(langs, langs2, label1 = "condition1", label2 = "condition2", saveto=None):
    DOT_COLOR1 = "lightblue"
    DOT_COLOR2 = "red"
    BG_COLOR   = "black" 
    df    = pd.DataFrame(langs.items(), columns=['Languages', 'Count'])
    df2   = pd.DataFrame(langs2.items(), columns=['Languages', 'Count'])
    
    plt.figure(figsize=(10,15)) 
    
    sb.stripplot(x='Count', y='Languages', data=df, \
                 size=5, color=DOT_COLOR1, label="have worked with", jitter=True)
    sb.stripplot(x='Count', y='Languages', data=df2, \
                 size=5, color=DOT_COLOR2, label="want to work with", jitter=True)
    
    # chatgpt draws my legend
    # Create custom legend handles to avoid duplicates
    # color = 'w' means do not draw line bissecting point
    blue_patch = plt.Line2D(
        [0], [0], marker='o', color=BG_COLOR, \
        label=label1, markerfacecolor=DOT_COLOR1, markersize=10)
    red_patch = plt.Line2D(
        [0], [0], marker='o', color=BG_COLOR, \
        label=label2, markerfacecolor=DOT_COLOR2, markersize=10)
    
    # Show the legend with custom handles
    plt.legend(handles=[blue_patch, red_patch], loc="center right")
    
    plt.grid(axis='x', linestyle='--', alpha=0.75) 
    plt.title("%s vs %s" % (label1, label2))
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    del df, df2

l1 = get_langs( so_df )
l2 = get_langs( so_df, "LanguageWantToWorkWith" )
visualize_langs(l1,l2, 
                label1="have worked with", label2="want to work with",
                saveto="images/used-vs-want2use.png")

l3 = get_langs( so_df, "LanguageAdmired")
l4 = get_langs( so_df, "LanguageWantToWorkWith")
visualize_langs(l3, l4, 
                label1="admired", label2="want to work with",
               saveto="images/admired-vs-want2use.png")
    

In [ ]:
# draw horizontal bar plot
# https://seaborn.pydata.org/examples/part_whole_bars.html

# investigate extrinsic vs intrinsic motivation
def get_difference(dict1, dict2, proportion=False):
    keys = dict1.keys()
    result = dict()
    for key in keys:
        if proportion:
            result[key] = round((dict1[key] - dict2[key])/dict2[key],2)
        else:
            result[key] = dict1[key] - dict2[key]
    return result

def visualize_diff(diff_dict, color="lightblue", saveto=None):
    diff_sorted = dict(
        sorted(diff_dict.items(), key=lambda item: item[1], reverse=True)
    )
    KEY = "Value"
    df    = pd.DataFrame(diff_sorted.items(), columns=['Languages', 'Value'])
    plt.figure(figsize=(15,20)) 
    sb.barplot(x=KEY, y='Languages', data=df, color=color)
    DELTA =  '\u0394'
    for index, value in enumerate(df[KEY]):
    # chatgpt annotates my chart
    # Position the text at the base of the bar
        if value >= 0:
            # Adjust the x position for positive values
            plt.text(value, index, DELTA+str(value), va='center', ha="left")  
        else:
             # Adjust the x position for negative values
            plt.text(value, index,  DELTA+str(value), va='center',  ha='right') 
    lowest = 0
    offset = 0
    positive_values = df[df[KEY] > 0][KEY]
    if not positive_values.empty:
        lowest = positive_values.min()
        offset = list(positive_values).count(lowest) 
    if len(positive_values) < len(df):
        # don't draw the line if every value is greater than 0_
        plt.axhline(y=df[KEY].tolist().index(lowest) + (offset-0.5), 
                    color='red', linestyle='--', zorder=-1)
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    
motiv_diff = get_difference(l2, l1, proportion=True)
# print(motiv_diff)
visualize_diff(motiv_diff, saveto="images/delta.png")
motiv_diff = get_difference(l2, l1)
visualize_diff(motiv_diff, saveto="images/delta-b.png")

# no clear description of what "admired" is
# in the schema
# but generally people want to use the languages
# they admire

# determine level of hype
# hype = get_difference(l4, l3)
# print(hype)
# visualize_diff(hype, color="red")

In [ ]:
# do people fall out of love with langs
# the more they are used professionally?

def visualize_favor(df, key_x, key_y, MAGIC_X=0, MAGIC_Y=0, title=str(), saveto=None):
    plt.figure()
    OFFSET = 1 # push text away from point slightly
    for i in range(merged.shape[0]):
        # label points that aren't un a cluster
        if merged[key_x][i] > MAGIC_X or merged[key_y][i] > MAGIC_Y:
            plt.text(merged[key_x].iloc[i]+OFFSET, 
                     merged[key_y].iloc[i]+OFFSET, 
                     merged["Language"].iloc[i], 
                     ha="left",
                     size='medium')

    sb.scatterplot(data=merged, x=key_x, y=key_y, hue="Language")
    plt.legend(loc='lower left', bbox_to_anchor=(0, -1.25), ncol=3) 
    plt.title(title)
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    pass
key_x  = "Users"
key_y  = "Potential '\u0394'Users"
df1    = pd.DataFrame(l1.items(), columns=['Language', key_x])
df2    = pd.DataFrame(motiv_diff.items(), columns=['Language', key_y])
# chatgpt tells me how to combine df
merged = pd.merge(df1, df2[["Language", key_y]], on='Language', how='left')
visualize_favor(merged, key_x, key_y, 
                MAGIC_X=5000, MAGIC_Y=2000, 
                saveto="images/favor.png")
del df1, df2, merged

In [ ]:
# see how much money are people making

def get_mean_by_category(df, category, key="ConvertedCompYearly"):
    unique = df[category].unique()
    result = dict()
    for u in unique:
        mean = df[df[category] == u][key].mean()
        result[u] = mean
    return result

def show_me_the_money(df, saveto=None):
    key_x = "ConvertedCompYearly"
    key_y = "DevType"
    
    means   = get_mean_by_category(df, key_y) 
    mean_df = pd.DataFrame(means.items(), columns=[key_y, key_x])

    plt.figure(figsize=(14,18)) 
    plt.axvline(x=1e5, color='red', linestyle='--', label="x = $100,000")
    plt.axvline(x=1e6, color='lightgreen', linestyle='--', label="x = millionaire")
    sb.barplot(x=key_x, y=key_y, data=mean_df.sort_values(by=key_x), \
               color='lavender', alpha=0.7, label="average compensation")
    sb.stripplot(x=key_x, y=key_y, data=df, \
                 size=3, jitter=True)
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    
# print survey ans
#employment_status = Counter(so_df["MainBranch"])
#print(employment_status)

#employment_type = Counter(so_df["DevType"])
#print(employment_type)

key = "ConvertedCompYearly"
#    answers = so_df[:-1][key].count()
#    print(answers, "people answered re: ", key)
df_no_na = so_df.dropna(subset=[key])
indices  = df_no_na[key].nlargest(15).index

show_me_the_money( df_no_na.drop(indices), saveto="images/compensation-by-profession.png" )
# could also ask myself what portion of developers 
# earn less than the mean compensation
# (what titles have high standard deviations in earnings)

In [ ]:

# key   = "DevType"
# prof  = "Developer, full-stack"

key   = "MainBranch"
prof = "I am a developer by profession"
col   = "ConvertedCompYearly"

devs =  df_no_na[df_no_na[key] ==  prof ] 
pd.set_option('display.float_format', '{:.2f}'.format)
devs.describe()[col]

# who the hell is making $1/yr 
# devs[devs[col] == 1.0]

# who are the millionaires
# devs[devs[col] > 1e6]

# who make more than the mean
# devs[devs[col] > 76230.84]

# who make more than the median
# devs[devs[col] > 63316.00]

# the ancient ones
so_df[so_df["YearsCodePro"] == 'More than 50 years']
# should drop the 18-24 year old who is either bullshitting or recalls a past life
# 55-64 years old
# 65 years or older